## SQLAlchemy

- [SQLAlchemy - The Database Toolkit for Python](https://www.sqlalchemy.org/)
    - Database を python コードで操作することが出来るライブラリ
- ORM Object Relational mpping
    - データベースを仮想的なオブジェクトにして、オブジェクト言語から扱う事が出来るようにする技法
    - SQLAlchemy は、Python のための ORM


### DataBase
- DataBase とは
    - キチンと整理された形式で保存されているデータの集まりのこと
    - そのようにデータを扱う事が出来るソフトウェアのこともデータベースと呼ぶことが多い
- データベースの種類
    - ![](./img/db.png)
    - (出展：[データベースとは？基礎知識を初心者にわかりやすく解説！｜ITトレンド](https://it-trend.jp/database/article/89-0065#chapter-1))
    - 今回使うのは RDB 
- Table とは
    - エクセルでいうシート部分
    - 項目とそのデータで構成されている
- 作成する table について
    - ![](./img/er.jpg)
- 今回どうしてDatabaseを使う必要があるのか

#### Engine
- Database へ接続するためのオブジェクト
- データベースへのパス、デバグのためのログ出力などの設定を行い、オブジェクトを作成する

In [ ]:
# sqlite ファイルへのパス。sqlite:/// の後に相対パスを渡す
DBFILE = "sqlite:///./SQLtest.db"

In [ ]:
# engine オブジェクトを作るためのメソッドをimport
from sqlalchemy import create_engine


In [ ]:
# engine オブジェクト作成
engine = create_engine(
    DBFILE, # db path 
    echo=True, # ログ出力
    connect_args={"check_same_thread": False} # SQLiteだけでの設定
)

`check_same_thread` についてはこちら：
[Threading/Pooling Behavior　SQLite — SQLAlchemy 1.4 Documentation](https://docs.sqlalchemy.org/en/14/dialects/sqlite.html?highlight=check_same_thread#threading-pooling-behavior)


#### Session
- ORM を使うときの基本的なトランザクション／データベース対話型オブジェクト
- 実際のデータ転送の開始から終了まで
- [sessionmaker](https://docs.sqlalchemy.org/en/14/orm/session_api.html?highlight=sessionmaker#sqlalchemy.orm.sessionmaker)

In [ ]:
# session オブジェクトを作るために、 sessionmaker をインポート
from sqlalchemy.orm import sessionmaker


SessionLocal = sessionmaker(
    bind=engine,     
    autocommit=False, 
    autoflush=False, # クエリの結果を自動的に得る。CommitせずにDBへ反映させるために使う。使う時は autocommit = Falseにしておく
)

In [ ]:
# session を作るクラスが生成された
type(SessionLocal)

- [declarative_base()](https://docs.sqlalchemy.org/en/14/orm/mapping_api.html#sqlalchemy.orm.declarative_base)
    - テーブルオブジェクトを作成ための基底クラスを構築するための関数
    - できたクラスが、`metadata` というクラスを持ち、これにテーブルオブジェクトを追加してDBへのコミットを行えるようにする
    - このクラスを継承してテーブルクラスを作成する

In [ ]:
# declarative_base をインポート 
from sqlalchemy.ext.declarative import declarative_base

# 慣習的に Base という名前でテーブルオブジェクトを作成
Base = declarative_base()


#### テーブルクラス作成

![](./image/er.jpg)


In [ ]:
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
class StatusCode(Base):
    __tablename__ = "statuscodes"
    id = Column(Integer, primary_key=True, index=True)
    code = Column(Integer, unique=True) # unique であれば Foreignkeyでつなげられる
    message = Column(String)

    # Cat クラスの statuscode プロパティに関連付けることによって、
    # StatusCode データオブジェクトを作った時に、
    # cats 属性を通じて、紐付いているCat のデータにアクセス出来る
    # 例えば、 status404.cats で、Cat 側が 外部キー(statuscodes.code) で紐づけているデータにアクセス出来る

    cats = relationship("Cat", back_populates="statuscode")

In [ ]:
class Cat(Base):
    __tablename__ = "cats"
    id = Column(Integer, primary_key=True, index=True)
    filepath = Column(String)
    message = Column(String)
    code = Column(Integer, ForeignKey("statuscodes.code"))

    # Status クラスの cats プロパティに関連付けることによって、この属性(statuscode) を通じて、Status のデータに catデータオブジェクトからアクセス出来る
    # 例えば、 a_cat という Catクラスのオブジェクトがあったとしたら、
    # a_cat.statuscode で、外部キー(statuscodes.code) で紐づけられている Status のデータをリストで取得出来る
    statuscode = relationship("StatusCode", back_populates="cats")


#### relationship

`relationship("関連付けたいクラス名文字列", back_populates="関連先のプロパティ名文字列")` 



#### DB設定とテーブルクラス作成の順番とまとめ

1. DBシステム部分の作成
    1. engine 作成 : DBへの接続担当
    1. session 作成 : DB への問い合わせ担当
1. テーブル作成
    1. テーブルクラスを作るための関数 `declarative_base()` で基底テーブルクラス作成
    1. 1 を継承して各テーブルを作成
    1. relationship があれば定義
1. 実際の接続
    1. `Base.metadata.create_all(bind=engine)` 

In [ ]:
# データベース作成、接続、テーブル作成。
# この部分を実行して初めてDBが作成される。engine 作成時に echo=True, にしているのでログが出力される
Base.metadata.create_all(bind=engine)


#### Transaction
- クエリなど1件ずつの問い合わせのこと
- 上記のログでいえば↑の一行がトランザクション


#### DB Browser for SQL で確認


### Data Insert

1. セッションオブジェクト `db` 作成 (セッションだけど、データベースとのやり取りに使うので、変数名は `db` がよく使われる。`session` もよく見る)
1. table data でデータオブジェクト `obj` を作成
1. `db.add(obj)` で追加。（**クエリに追加しただけ**でDBへのインサートはまだ行われない）
1. `db.commit()` でDBに初めて追加される
1. `db.refresh(obj)` で obj へ SELECT されたデータが入る（←かなり肝）


In [ ]:
# 上記で生成した、SessionLocal クラスを使って、Sessionオブジェクトを作成
db = SessionLocal()

In [ ]:
# データオブジェクト作成
# id は primary key なので、DBで連番が自動で割り振られるので渡さない
status_404 = StatusCode(code=404, message="Not Found")
status_200 = StatusCode(code=200, message="OK")


In [ ]:
# クエリを追加
db.add(status_404)
db.add(status_200)


In [ ]:
# DBへデータをInsert
db.commit() 

In [ ]:
# マジック部分。
# refresh すると、SELECT 文が走って、データをDBからフェッチし、テーブルオブジェクトに反映される。
db.refresh(status_404)
db.refresh(status_200)

# status_404.id 

In [ ]:
# Selectして取ってきたデータを確認。IDも得た
status_404.__dict__


- 演習: 以下のデータで cat table object を作ってDBへ挿入して下さい（メッセージ部分は好きに変えていいですよ）
    1.  code 200 / message "やったね！" / filapth "https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png"
    1. code 400 / message "てへぺろ" / filepath "https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png"


In [ ]:
# cat
cat_200 = Cat(
    filepath="https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png", 
    message="やったね", 
    code=200)

cat_404 = Cat(
    filepath="https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png", 
    message="てへぺろ", 
    code=404)

In [ ]:
db.add(cat_200)
db.add(cat_404)
db.commit()
db.refresh(cat_200)
db.refresh(cat_404)

### SELECT

セッションオブジェクト (`db`) が持つ、`.query()` メソッドを使います

1. db.query(`テーブルクラス`) で `sqlalchemy.orm.query.Query` オブジェクトを得る
1. query オブジェクトが持つメソッドでデータを得る

- [Query API — SQLAlchemy 1.4 Documentation](https://docs.sqlalchemy.org/en/14/orm/query.html?highlight=query#sqlalchemy.orm.Query)


In [ ]:
type(db.query(StatusCode))


In [ ]:
# .all() メソッドで、StatusCodeテーブルに入っている全データの1つずつを query object として、リストで得る
all_status = db.query(StatusCode).all()


In [ ]:
# コラム名でデータを取得可
for status in all_status:
    print(status.id, status.code, status.message)



In [ ]:
cats = db.query(Cat).all()
for cat in cats:
    print(cat.id, cat.code, cat.message, cat.filepath)


### SELECT by ID 

- queryオブジェクトが持つ `.get()` メソッド で primary key を指定してデータを取得出来ます
- [method sqlalchemy.orm.Query.get](https://docs.sqlalchemy.org/en/14/orm/query.html?highlight=query#sqlalchemy.orm.Query.get)

In [ ]:
a_cat = db.query(Cat).get(1)
print(a_cat, a_cat.id)

### SELECT with filter 

`SELECT * FROM cats WHERE ~~~ ` のような条件クエリは queryオブジェクトが持つ `filter()` メソッドを使います

- db.query(`テーブル名`).filter(`テーブル名.プロパティ` == ?)
- これでクエリオブジェクトが返る

In [ ]:
cat_filted_by_404 = db.query(Cat).filter(Cat.code == 404)
type(cat_filted_by_404)

In [ ]:
# 最初の結果だけ得たい場合は、 first() メソッドを使う
a_cat = cat_filted_by_404.first()
print(a_cat.id, a_cat.message)

### UPDATE
- query オブジェクトで得たデータを直接書き換えたあと、session を commit する

In [ ]:
# .first() まで一度に書くのもあり
cat_filted_by_404 = db.query(Cat).filter(Cat.code == 404).first()
cat_filted_by_404.message = "やっちゃったてへぺろ"
db.commit()
db.refresh(cat_filted_by_404)

print(cat_filted_by_404.id, cat_filted_by_404.message)


### Relation

Cat テーブル定義で、`"statuscodes.code"` へ外部キーを貼ることで、StatusCode の code を参照しています。

この状態で、`relationship` をお互いに張っているので、当該プロパティから、相手側のデータにアクセス出来ます

```python

class StatusCode(Base):
    __tablename__ = "statuscodes"
    # (略)
    cats = relationship("Cat", back_populates="statuscode")

class Cat(Base):
    __tablename__ = "cats"
    # (略)
    code = Column(Integer, ForeignKey("statuscodes.code"))
    statuscode = relationship("StatusCode", back_populates="cats")
```


In [ ]:
# 200番のステイタスデータを取得
status_200 = db.query(StatusCode).filter(StatusCode.code == 200).first()


In [ ]:
# このクエリオブジェクトは、cats プロパティを通じて、200番の `code` を持つ cat データにアクセス可
status_200.cats


In [ ]:
# 200番 cat を追加でインサート
new_cat_200 = Cat(
    filepath="http://2.bp.blogspot.com/-5RqOJ4QvbXo/VEETRWSWxKI/AAAAAAAAocY/OAMZmQl4DPA/s180-c/cat_matatabi.png", 
    message="追加の200番キャットです", 
    code=200)

db.add(new_cat_200)
db.commit()


In [ ]:
# statuscode オブジェクトの .cats メソッドを確認。ちゃんと1匹増えました
status_200.cats

In [ ]:
for cat in status_200.cats:
    print (cat.code, cat.message)